In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor
from sklearn import neighbors
from sklearn import svm
import xgboost as xgb


from sklearn import preprocessing
import math


In [2]:

%matplotlib inline
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200

In [4]:
df_train = pd.read_csv("data\\train.csv")
df_test = pd.read_csv("data\\test.csv")
df_mck = pd.read_excel("data\\MCK.xlsx")
df_columns = pd.read_excel("data\\column_comparison.xlsx")
df_country = pd.read_excel("data\\country_mapping.xlsx")

df = pd.concat([df_train, df_test], keys=['train', 'test'])
# df = df_train

In [ ]:
# df_columns = df_columns.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop(["Intergalactic Development Index (IDI), male, Rank"],axis=1)
df = df.drop(["Intergalactic Development Index (IDI), female, Rank"],axis=1)
df = df.drop(["Intergalactic Development Index (IDI), Rank"],axis=1)

In [ ]:
df_train.shape , df_test.shape

In [ ]:
"""
@params:
    df=dataframe
    targetVariable = Target variable
    numberOfColumns = number of values against targetVariable should be compared with (top k variables with highest correlation)
    largestCorrelation : true if values with largest corrleation is wanted and false otherwise
"""
def getTopCorrelationColumnsNames(df, targetVariable, numberOfColumns, largestCorrelation):
    corrmat = df.corr()
    if largestCorrelation==True:
        return corrmat.nlargest(numberOfColumns, targetVariable)[targetVariable].index
    else:
        return corrmat.nsmallest(numberOfColumns, targetVariable)[targetVariable].index
    

In [ ]:
topColumns = getTopCorrelationColumnsNames(df_train,"y",31,True).tolist()
len(topColumns)
df_train_short = df_train[topColumns]


In [ ]:
# df_train_short.shape,df_train_short.isnull().any()

In [ ]:
df_train.shape , df_mck.shape , df_columns.shape , df_train_short.shape, df_country.shape

##### 1. combine train test
##### 2. rename trainTest
##### 3. find std dev of each feature for each country / galaxy on trainTest and mck data
##### 4. apply knn

In [ ]:
trainColumns = df_columns["TRAIN"].tolist()
trainMckColumns = df_columns["MCK"].tolist()

columnDictionary = {}
for train_col, trainMck_col in zip(trainColumns,trainMckColumns) :
    columnDictionary[train_col] = trainMck_col
#     {old_name : new name}

In [ ]:
galaxyColumns = df_country["galaxy"].tolist()
countryColumns = df_country["country"].tolist()

countryDictionary = {}
for galaxy_col, country_col in zip(galaxyColumns,countryColumns) :
    countryDictionary[galaxy_col] = country_col
#     {old_name : new name}

In [ ]:
df_renamed = df.rename(columns=columnDictionary,inplace=False)

In [ ]:
# confirm taking mean
originalDfColumns = df_renamed.columns.tolist()

df_CombinedGrouped = df_renamed.groupby("Country").mean()

df_mck = df_mck[ originalDfColumns[:-1]]
                       
df_mckGrouped = df_mck.groupby("Country").mean()
zColumns = [ (col_name+"_zscore") for col_name in df_CombinedGrouped.columns.tolist() ]

df_renamed_mean = df_renamed.mean()   #mean of  whole dataset for each feature
df_renamed_std = df_renamed.std()    #std of  whole dataset for each feature

df_mck_mean = df_mck.mean()   #mean of  whole dataset for each feature
df_mck_std = df_mck.std()    #std of  whole dataset for each feature

df_CombinedGrouped.shape, df_mckGrouped.shape

In [ ]:
#function to add columns 
def addColumn(df,columnName,columnValue=0):
    df[columnName] = columnValue
    return df

In [ ]:
# adding zscore column to the dataset and each will have a value of 0
for cols in zColumns:
    df_CombinedGrouped = addColumn(df_CombinedGrouped,cols,0.0)
    if "y_zscore" not in cols:
        df_mckGrouped = addColumn(df_mckGrouped,cols,0.0)

In [ ]:
for index, row in df_CombinedGrouped.iterrows():
    #   col_name is name of the column and its value is that column's mean
    for col_name in  df_renamed_mean.index.tolist() :
        if not math.isnan(row[col_name]):
            df_CombinedGrouped.at[index,(col_name+"_zscore")] = ( row[col_name] - df_renamed_mean[col_name] )  / df_renamed_std[col_name]

for index, row in df_mckGrouped.iterrows():
    #   col_name is name of the column and its value is that column's mean
    for col_name in  df_mck_mean.index.tolist() :
        if not math.isnan(row[col_name]):
            df_mckGrouped.at[index,(col_name+"_zscore")] = ( row[col_name] - df_mck_mean[col_name] )  / df_mck_std[col_name]
            

In [ ]:
df_CombinedGrouped = df_CombinedGrouped.iloc[:,-76:]
df_mckGrouped = df_mckGrouped.iloc[:,-75:]

In [ ]:
CountryNames_unqiue = list(set(df_mckGrouped.index.tolist()))

In [ ]:
df_mckGrouped.fillna(df_mckGrouped.mean(),inplace=True)
df_CombinedGrouped.fillna(df_CombinedGrouped.mean(),inplace=True)

df_mckGrouped = addColumn(df_mckGrouped,"Country_encoded",-1)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(CountryNames_unqiue)

In [ ]:
for index, row in df_mckGrouped.iterrows():
    df_mckGrouped.at[index,"Country_encoded"] = le.transform([index]).tolist()[0]

In [ ]:
X = df_mckGrouped.drop(['Country_encoded',"Year_zscore"],axis=1)
y = df_mckGrouped['Country_encoded']

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=1,algorithm="ball_tree")
classifier.fit(X, y)

In [ ]:
X_test = df_CombinedGrouped.drop(["y_zscore","Year_zscore"],axis=1)
# X_test = df_CombinedGrouped.drop(["y_zscore","Year_zscore","Predicted Country code"],axis=1)

# y_pred = classifier.predict(X_test)
df_CombinedGrouped["Predicted Country code"] = classifier.predict(X_test)

# y_pred = classifier.kneighbors(X_test,n_neighbors=2)

In [ ]:
# #FOR kneighbors and K=1
# outpred1 = []
# outpred2 = []


# for i in y_pred[0]:
#     outpred1.append(i[0])

# for i in y_pred[1]:
#     outpred2.append(i[0])



# ydict = {"outpred1":outpred1}

# y_pred = pd.DataFrame(ydict,outpred2)

In [ ]:
# #FOR kneighbors and K=2

# outpred1 = []
# outpred2 = []
# outpred3 = []
# outpred4 = []

# for i in y_pred[0]:
#     outpred1.append(i[0])

# for i in y_pred[0]:
#     outpred2.append(i[1])

# for i in y_pred[1]:
#     outpred3.append(i[0])

# for i in y_pred[1]:
#     outpred4.append(i[1])

# ydict = {"outpred1":outpred1,"outpred2":outpred2,"outpred4":outpred4}

# y_pred = pd.DataFrame(ydict,outpred3)

In [ ]:
# y_pred = pd.DataFrame(y_pred)
# y_pred[y_pred.duplicated()]
# y_pred[y_pred.index.duplicated()]
# y_pred.sort_values(by="outpred1",ascending=False)

In [ ]:
df_CombinedGrouped = addColumn(df_CombinedGrouped,"galaxyToCountry_junaid","")
df_CombinedGrouped = addColumn(df_CombinedGrouped,"Predicted Country","")

In [ ]:
for index,row in df_CombinedGrouped.iterrows():
    if index in countryDictionary:
        df_CombinedGrouped.at[index,"galaxyToCountry_junaid"] = countryDictionary[index]
    pred_country_code = row["Predicted Country code"]
#     if not pred_country_code== -1:
    df_CombinedGrouped.at[index,"Predicted Country"] = le.inverse_transform([pred_country_code]).tolist()[0]

In [ ]:
# finding difference between model predictions and junaid mapping
# wrongPredicted = 0
# for index,row in df_CombinedGrouped.iterrows():
#     if row["galaxyToCountry_junaid"] != row["Predicted Country"]:
#         wrongPredicted += 1
# wrongPredicted

In [ ]:
galaxyCountryModelMapping = {}
for index,row in df_CombinedGrouped.iterrows():
    galaxyCountryModelMapping[index] = row["Predicted Country"] 

In [ ]:
df_mck_copy = df_mck.copy()

In [ ]:
df_withModelCountry = df_renamed.copy()
# renaming country column to galaxy so it will be easier to merge mck and trainTest datasets
df_withModelCountry.rename(columns={"Country":"galaxy"},inplace=True)
df_withModelCountry = addColumn(df_withModelCountry,"Country","")

In [ ]:
for index,row in df_withModelCountry.iterrows():
    countryName = row["galaxy"]
    df_withModelCountry.at[index,"Country"] = galaxyCountryModelMapping[countryName]


In [ ]:
df_withModelCountry['Year'] = (df_withModelCountry['Year'].rank(method = 'dense') + 1989)
df_withModelCountry['Year'] = df_withModelCountry['Year'].astype(int)


In [ ]:
df_withModelCountry = df_withModelCountry[df_withModelCountry['Year']>2010]

In [ ]:
# df_withModelCountry = df_withModelCountry.astype(str)
# df_mck_copy = df_mck_copy.astype(str)


In [ ]:
dfTrainTestMck_merged = pd.merge(df_withModelCountry, df_mck_copy,  how='left', left_on=['Country','Year'], right_on = ['Country','Year'])



## We Have the Merged dataset. ^

In [ ]:
#taking year to the end for future processing
dfTrainTestMck_merged = dfTrainTestMck_merged[[c for c in dfTrainTestMck_merged if c not in ['Year']] 
       + ['Year']]

#merged, arranged (year country at start and y at end) , will be used for regression analysis
mck_merged = dfTrainTestMck_merged.iloc[:,75:]
mck_merged = mck_merged[ ['Year'] + [c for c in mck_merged if c not in ['Year']]  ]  # just taking year column at start of df
mck_merged = addColumn(mck_merged,"Country_encoded","")

for index, row in mck_merged.iterrows():
    countryName = row["Country"]
    mck_merged.loc[index,"Country_encoded"] = le.transform([countryName]).tolist()[0]

mck_merged = mck_merged[ [c for c in mck_merged if c not in ['y']] + ['y'] ]


In [ ]:
dfTrainTestMck_merged = dfTrainTestMck_merged[['Year'] + [c for c in dfTrainTestMck_merged if c not in ['Year']] ]
dfTrainTest_merged = dfTrainTestMck_merged.iloc[:,:78]

#below 2 lines are just arranging column positions
dfTrainTest_merged = dfTrainTest_merged[ ['Year','Country'] + [c for c in dfTrainTest_merged if c not in ['Year','Country']]  ]  # just taking year column at start of df

dfTrainTest_merged = addColumn(dfTrainTest_merged,"Country_encoded","")
for index, row in dfTrainTest_merged.iterrows():
    countryName = row["Country"]
    dfTrainTest_merged.at[index,"Country_encoded"] = le.transform([countryName]).tolist()[0]

dfTrainTest_merged = dfTrainTest_merged[ [c for c in dfTrainTest_merged if c not in ['y']] + ['y']   ]  # just taking year column at start of df
    

In [ ]:
X_train = mck_merged.drop(['y','Country'], axis=1)
y_train = mck_merged['y']
X_train.fillna(X_train.mean(),inplace=True)

X_test = dfTrainTest_merged.drop(['y','Country','galaxy'], axis=1)
y_test = dfTrainTest_merged['y']
X_test.fillna(X_test.mean(),inplace=True)


In [ ]:
# Linear Regression
reg = LinearRegression().fit(X_train, y_train)
y_pred_reg = reg.predict(X_test)

rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],0):
        correctCounter+=1
correctCounter

In [ ]:
# SCR
clf = svm.SVR( gamma='scale', C=1.0, epsilon=0.2)
clf.fit(X_train,y_train) 
y_pred_reg = clf.predict(X_test)

rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],0):
        correctCounter+=1
correctCounter

In [ ]:
# XGboost
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 1, learning_rate = 0.5,
                max_depth = 1000, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train.values,y_train)
y_pred_reg = xg_reg.predict(X_test.values)


rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],0):
        correctCounter+=1
correctCounter

In [ ]:
param = {'max_depth': 9, 'objective': 'reg:linear', 'base_score':0.2, 'booster':'gbtree', 'colsample_bylevel':1,
         'colsample_bytree':0.8}
param['nthread'] = 4
param['eval_metric'] = 'rmse'

In [ ]:
dtrain = xgb.DMatrix(X_train.values, label=y_train, missing=-999.0)
dtest = xgb.DMatrix(X_test.values, label=y_test, missing=-999.0)
evallist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 50
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
#for missing = -999 :                                             eval-rmse:0.119211	train-rmse:0.004722
# for missing 0 :                                                 eval-rmse:0.116161	train-rmse:0.004726
# for missing 0 and base_score:0.2:                               eval-rmse:0.084709	train-rmse:0.004723
# for missing -999 and base_score:0.2:                            eval-rmse:0.087144	train-rmse:0.00472
# for missing -999 and base_score:0.2:                            eval-rmse:0.087144	train-rmse:0.00472
# for missing -999 and base_score:0.2 and 'max_depth': 10:        eval-rmse:0.083773	train-rmse:0.004708


In [ ]:
# Nearest neighbours
knn = neighbors.KNeighborsRegressor(5)
knn.fit(X_train.values,y_train)
y_pred_reg = knn.predict(X_test.values)


rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],2):
        correctCounter+=1
correctCounter

In [ ]:
#ensemble
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=10)
reg2 = RandomForestRegressor(random_state=1, n_estimators=10)
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(X_train, y_train)

y_pred_reg = ereg.predict(X_test.values)


rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],2):
        correctCounter+=1
correctCounter

In [ ]:
# GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')
gbr.fit(X_train.values,y_train)
y_pred_reg = gbr.predict(X_test.values)


rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],2):
        correctCounter+=1
correctCounter

In [ ]:
gbr.set_params(n_estimators=200, warm_start=True)  # set warm_start and new nr of trees
gbr.fit(X_train, y_train) # fit additional 100 trees to est
y_pred_reg = gbr.predict(X_test.values)


rmse = (np.sqrt(mean_squared_error(y_test, y_pred_reg)))
r2 = r2_score(y_test, y_pred_reg)
adjR2 = 1-(1-r2)*((X_train.shape[0])-1)/((X_train.shape[0])-(X_train.shape[1])-1)

print("RMSE: %f" % rmse)
print("R2 score = ",r2)        # r2 score 
print("Adjusted R2 score = ",adjR2)

correctCounter = 0
for i in range(len(y_test)):
    if round(y_test[i],1) == round(y_pred_reg[i],2):
        correctCounter+=1
correctCounter

In [ ]:
# df_test = addColumn(df_test,"y","0")

In [ ]:
df_test_renamed = df_test.rename(columns=columnDictionary,inplace=False)
df_test_renamed = addColumn(df_test_renamed,"Country Mapped","")

In [ ]:
df_test_renamed = addColumn(df_test_renamed,"Country_encoded",-1)

In [ ]:

for index,row in df_test_renamed.iterrows():
    countryName = row["Country"]
    df_test_renamed.at[index,"Country Mapped"] = galaxyCountryModelMapping[countryName]


In [ ]:
for index, row in df_test_renamed.iterrows():
    countryName = row["Country Mapped"]
    df_test_renamed.at[index,"Country_encoded"] = le.transform([countryName]).tolist()[0]


In [ ]:
df_test_renamed.fillna(df_test_renamed.mean(),inplace=True)

In [ ]:
df_test_renamed['YearRanked'] = (df_test_renamed['Year'].rank(method = 'dense') + 1989)
df_test_renamed['Year'] = df_test_renamed['Year'].astype(int)


In [ ]:
# df_columns = df_columns.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_test_renamed = df_test_renamed.drop(["Intergalactic Development Index (IDI), male, Rank"],axis=1)
df_test_renamed = df_test_renamed.drop(["Intergalactic Development Index (IDI), female, Rank"],axis=1)
df_test_renamed = df_test_renamed.drop(["Intergalactic Development Index (IDI), Rank"],axis=1)

In [ ]:
X_test_test = df_test_renamed.drop(['Country','Country Mapped',"Year"], axis=1)


In [ ]:

# example :
# fileName = "pred.txt"
# mylist = [1,2,3,4]

def writeTofile(fileName,mylist):
    with open(fileName, 'w') as f:
        for item in mylist:
            f.write("%s\n" % item)

In [ ]:
y_pred_reg = gbr.predict(X_test_test.values)
pd.DataFrame(y_pred_reg)


In [ ]:
y_pred_reg[0]

In [7]:
len(saadpred)

890

In [11]:
gs = pd.DataFrame(saadpred)
gs.to_excel("saadpred.xlsx")